In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


0f85060ff61081cfe4a6c1b1439514a3

In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset/download?datasetVersionNumber=2')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: johndiale
Your Kaggle Key: ··········


100%|██████████| 11.4M/11.4M [00:00<00:00, 83.3MB/s]

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

# Load data
data = pd.read_csv('ag-news-classification-dataset/train.csv')

# Tokenize text
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['Title'] + ' ' + data['Description'])
X = tokenizer.texts_to_sequences(data['Title'] + ' ' + data['Description'])

# Pad sequences
max_len = 100
X = pad_sequences(X, maxlen=max_len, padding='post', truncating='post')

# Encode class labels
y = np.array(data['Class Index'] - 1)  # Subtract 1 to get 0-based class indices
num_classes = len(np.unique(y))
y = np.eye(num_classes)[y]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D

# Define model architecture
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=32, input_length=100))
model.add(Conv1D(64, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))


# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5
1500/1500 [==============================] - 185s 116ms/step - loss: 0.6499 - accuracy: 0.7195 - val_loss: 0.3427 - val_accuracy: 0.8970
Epoch 2/5
1500/1500 [==============================] - 156s 104ms/step - loss: 0.2898 - accuracy: 0.9118 - val_loss: 0.2965 - val_accuracy: 0.9038
Epoch 3/5
1500/1500 [==============================] - 152s 101ms/step - loss: 0.2422 - accuracy: 0.9258 - val_loss: 0.2885 - val_accuracy: 0.9094
Epoch 4/5
1500/1500 [==============================] - 151s 101ms/step - loss: 0.2087 - accuracy: 0.9357 - val_loss: 0.2749 - val_accuracy: 0.9118
Epoch 5/5
1500/1500 [==============================] - 156s 104ms/step - loss: 0.1795 - accuracy: 0.9456 - val_loss: 0.2819 - val_accuracy: 0.9104


In [ ]:
model.save('news-classification-model.h5')

In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.9 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=a8d3c182121709174f81afdfbf6df7ccfcb7c0baeb5e4b2fe666ba15d9a077d7
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [ ]:
import streamlit as st
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load model
model = load_model('news-classification-model.h5')

# Load tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['Title'] + ' ' + data['Description'])

# Define function to make prediction
def predict_class(title, description):
    # Tokenize input
    text = title + ' ' + description
    x = tokenizer.texts_to_sequences([text])
    x = pad_sequences(x, maxlen=100, padding='post', truncating='post')
    # Make prediction
    y_pred = model.predict(x)
    # Decode prediction
    class_names = ['World', 'Sports', 'Business', 'Sci/Tech']
    class_id = y_pred.argmax(axis=-1)[0]
    class_name = class_names[class_id]
    return class_name

# Define Streamlit app
st.title('News Article Classifier')
title = st.text_input('Title')
description = st.text_area('Description')
if st.button('Classify'):
    class_name = predict_class(title, description)
    st.write('Predicted class:', class_name)


2023-03-27 14:20:21.664 WARNING tensorflow: Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
  command:

    streamlit run /usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2023-03-27 14:20:22.126 
  command:

    streamlit run /usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py [ARGUMENTS]
